In [12]:
import time
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix
import seaborn as sns
import seaborn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [13]:
writer = SummaryWriter()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [14]:
def load_Data():
    
    train_dataset = datasets.ImageFolder(r'C:\Users\Shree\Desktop\AI_PROJECT\Animal Classification\Final_Dataset\dataset1\train', transform=transform)
    val_dataset = datasets.ImageFolder(r'C:\Users\Shree\Desktop\AI_PROJECT\Animal Classification\Final_Dataset\dataset1\val', transform=transform)
    test_dataset = datasets.ImageFolder(r'C:\Users\Shree\Desktop\AI_PROJECT\Animal Classification\Final_Dataset\dataset1\test', transform=transform)

    #print(train_dataset.class_to_idx)
    #print(test_dataset.class_to_idx)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    return train_loader,val_loader,test_loader

In [15]:
def Model():
    model = torchvision.models.mobilenet_v2(pretrained=False)
    num_ftrs = model.classifier[1].in_features
    #print(num_ftrs)
    model.classifier = nn.Sequential(
    nn.Linear(num_ftrs,640),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.1),
    
    nn.Linear(640,320),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.1),
    
    nn.Linear(320,160),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.1),
    
    nn.Linear(160, 2)
    )
    return model.to(device)
Model()

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [16]:
def save_checkpoint(state, filename):
    torch.save(state, filename)


In [17]:
y_train = []
y_expected_train = []
y_val = []
y_expected_val = []
y_test = []
y_expected_test = []

In [18]:
def train(model, train_loader, criterion, optimizer, epoch, step=5):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        #print(torch.cuda.is_available())
        total =0
        correct = 0
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        #print(predicted,type(predicted))
        #print(labels,type(labels))
        output = model(images)
        
        loss = criterion(output,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(output.data, 1)
        y_train.extend(predicted.to(torch.device('cpu')))
        y_expected_train.extend(labels.to(torch.device('cpu')))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        #print( f'Training Accuracy: {accuracy:.2f}%')
            
        if i%step==0:
            print('EPOCH {} | ITER {} | AVG_LOSS {} | Train_ACC {}'.format(epoch, i, loss,accuracy))
        writer.add_scalar('TRAIN_LOSS', loss, epoch)
        
    return loss ,accuracy

In [19]:
def val(model, test_loader, criterion, optimizer, epoch, step=5):
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            #print(inputs.shape)
            #print(labels.shape)
            labels = labels.to(device)
            outputs = model(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            #print(predicted,type(predicted))
            #print(labels,type(labels))
            y_val.extend(predicted.to(torch.device('cpu')))
            y_expected_val.extend(labels.to(torch.device('cpu')))
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            accuracy = 100 * correct / total
            print(f'Val Accuracy: {accuracy:.2f}%')
            return accuracy
    

In [20]:
def test(model, test_loader, criterion, optimizer, epoch, step=5):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            #print(inputs.shape)
            #print(labels.shape)
            labels = labels.to(device)
            outputs = model(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            #print(predicted,type(predicted))
            #print(labels,type(labels))
            y_test.extend(predicted.to(torch.device('cpu')))
            y_expected_test.extend(labels.to(torch.device('cpu')))
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            accuracy = 100 * correct / total
            print(f'Test Accuracy: {accuracy:.2f}%')
            return accuracy
    

In [ ]:
def main():
    model = Model()
    print(torch.cuda.is_available())
    if torch.cuda.is_available():
        model = model.cuda()
        print("using gpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    epochs = 20

    train_loader, val_loader ,test_loader = load_Data()
    
    history = open(r'.\Checkpoint1\history.csv','w')
    history.write('epochs , trainloss , val_acc , test_acc \n')

    for epoch in range(0,epochs):
#         print(epoch)
        start = time.time()
        train_loss,accuracy = train(model, train_loader, criterion, optimizer, epoch)
        val_loss = val(model, val_loader, criterion, optimizer, epoch)
        test_loss = test(model, test_loader, criterion, optimizer, epoch)
#         val_loss = train(model, val_loader, criterion, optimizer, epoch)
        print()
        print('-' * 50)
        print('EPOCH {} | LOSS {} | TIME {}'.format(epoch, train_loss, time.time() - start))
        print('-' * 50)
        print()


        history.write('{},{},{},{},{}\n'.format(epoch, train_loss,accuracy, val_loss , test_loss))
        save_checkpoint({'epoch': epoch,'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict(),'loss' : train_loss}, r'.\Checkpoint1\checkpoint_{}.ckpt'.format(epoch))
    history.close()
    

# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    main()

False
EPOCH 0 | ITER 0 | AVG_LOSS 0.7009494304656982 | Train_ACC 46.875
EPOCH 0 | ITER 5 | AVG_LOSS 0.6753954887390137 | Train_ACC 56.25
EPOCH 0 | ITER 10 | AVG_LOSS 0.6931236386299133 | Train_ACC 53.125
EPOCH 0 | ITER 15 | AVG_LOSS 0.6917001605033875 | Train_ACC 53.125
EPOCH 0 | ITER 20 | AVG_LOSS 0.6820389628410339 | Train_ACC 62.5
EPOCH 0 | ITER 25 | AVG_LOSS 0.7241309881210327 | Train_ACC 46.875
EPOCH 0 | ITER 30 | AVG_LOSS 0.680141031742096 | Train_ACC 62.5
EPOCH 0 | ITER 35 | AVG_LOSS 0.6256160736083984 | Train_ACC 56.25
EPOCH 0 | ITER 40 | AVG_LOSS 0.650295078754425 | Train_ACC 68.75
EPOCH 0 | ITER 45 | AVG_LOSS 0.6570795178413391 | Train_ACC 62.5
EPOCH 0 | ITER 50 | AVG_LOSS 0.6937109231948853 | Train_ACC 59.375
EPOCH 0 | ITER 55 | AVG_LOSS 0.678638756275177 | Train_ACC 56.25
EPOCH 0 | ITER 60 | AVG_LOSS 0.6401710510253906 | Train_ACC 68.75
EPOCH 0 | ITER 65 | AVG_LOSS 0.5557555556297302 | Train_ACC 78.125
EPOCH 0 | ITER 70 | AVG_LOSS 0.6394736170768738 | Train_ACC 62.5
EPOCH 0

In [ ]:
def visualize(model , data_loader):
    class2idx = {'Dog': 0, 'Cat': 1}
    idx2class = { value : key for key,value in  class2idx.items() }
    embeddings = np.array([])
    labels = np.array([])
    model.eval()
    out_features = 1280
    num_categories = 2
    with torch.no_grad(): 
        for i,data in enumerate(data_loader):
            images, labs = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            embeds = model.features["embeddings"].reshape(-1, out_features)
            embeddings = np.append(embeddings , embeds)
            labels = np.append(labels,labs.cpu())
    
    model.train()
    embeddings = embeddings.reshape(-1, 1280)
    print(embeddings)
    
    c_labels = labels.reshape(-1 , 1).ravel()
    tsne = TSNE(n_components=2).fit_transform(embeddings)
    labels = np.array([ idx2class[ele] for ele in c_labels])
    
    cmap = cm.get_cmap('tab20')
    
    for lab in range(num_categories):
        indices = (c_labels == lab)
        plt.scatter(tsne[indices, 0],tsne[indices, 1],c=np.array(cmap(lab)).reshape(1, 4),label=lab,alpha=1)
        plt.legend(fontsize='large', markerscale=2)
    plt.show()
   

In [ ]:
model = Model()
train_loader, val_loader ,test_loader = load_Data()
visualize(model,train_loader)

In [ ]:
print(y_test) 
print(y_expected_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import seaborn
import matplotlib.pyplot as plt

def plot_confusion_matrix(data, labels, output_filename=None):

    seaborn.set(color_codes=True)
    plt.figure(1, figsize=(9, 6))
 
    plt.title("Confusion Matrix - FOOD101 - DENSENET")
 
    seaborn.set(font_scale=1.4)
    ax = seaborn.heatmap(data, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Scale'})
 
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
 
    ax.set(ylabel="True Label", xlabel="Predicted Label")
    
    plt.savefig(output_filename, bbox_inches='tight', dpi=300)
    plt.show()
    plt.close()

class2idx = {'cheesecake': 0, 'french_fries': 1, 'fried_rice': 2, 'garlic_bread': 3}

idx2class = { value : key  for key , value in class2idx.items() }
 
print(y_expected[0])

y_true = [  idx2class[val.item()] for val in y_expected ]
y_pred = [ idx2class[val.item()] for val in y_test ]


print(y_true[:10])
print(y_pred[:10])

cnf_matrix = confusion_matrix(y_true , y_pred , labels=['cheesecake', 'french_fries', 'fried_rice', 'garlic_bread'])

plot_confusion_matrix(cnf_matrix , ['cheesecake', 'french_fries', 'fried_rice', 'garlic_bread'] , output_filename = "confusion_matrix.png")